In [ ]:
import requests 
from bs4 import BeautifulSoup
import re
import os

In [ ]:
input_keyword = '網路爬蟲'
utf8_url = repr(input_keyword.encode('UTF-8')).upper()
utf8_url = utf8_url.replace('\\X','%')
#print(utf8_url)
print("%s: %s" % (input_keyword, utf8_url[2:-1:1])) 

wiki_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(wiki_keyword_link)

In [ ]:
#模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + wiki_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + wiki_keyword_link
resp = requests.get(url,headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text,'lxml')
content = html.find('div',attrs={'id':'mw-content-text'}).find_all('p')

for paragraph in content:
    print(paragraph.text)

In [ ]:
for ext_link in content:
    a_tag = ext_link.find_all('a',href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_str in a_tag:
            a_link = link_str['href']
            a_key_word = link_str.text
            print('外部連結: [%s] %s'%(a_key_word,a_link))

In [ ]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #

        titleName = html.find('h1',{'id':'firstHeading'}).text
        if '/' in titleName:
            titleName = titleName.replace('/','')
        #print(titleName)
        path = "./WikiArticle"
        if not os.path.isdir(path):
            os.mkdir(path)
        
        with open(r"{0}/{1}.txt".format(path,titleName), "w",encoding='utf-8') as fw:
            for ct in content:
                #print(ct.text)
                fw.write(str(ct))
        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        with open("{0}/{1}.txt".format(path,titleName), "r",encoding='utf-8') as fr:
            file = fr.read()
            soup = BeautifulSoup(file, "lxml")
            items = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
            for item in items:
                links = item.find_all('a',href=re.compile("^(/wiki/)((?!;)\S)*$"))
                for link in links:
                    if 'title' in link.attrs:
                        #print(link['title'])
                        #print(link['href'])
                        #print('=====================')
                        external_link_dict[link['href']] = link['title'] 
        #print(external_link_dict)         
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程